# Finding Price Elasticity of Demand

In [12]:
import pandas as pd
import datetime
import numpy as np
data = pd.read_csv("Cleaned_Data2013-14.csv")
df_concat = data[['Unnamed: 0', 'Commodity', 'State', 'Market Center', 'Report Date', 'Modal Prices(in Rs/quintals)', 'Arrival_Tonnes_Calc']]
df_concat.head()

,Unnamed: 0,Commodity,State,Market Center,Report Date,Modal Prices(in Rs/quintals),Arrival_Tonnes_Calc
0,0,Coffee,Kerala,Kalpetta,20130403,12000.0,0.5
1,1,Coffee,Kerala,Kalpetta,20130404,12000.0,0.5
2,2,Coffee,Kerala,Kalpetta,20130405,12000.0,0.5
3,3,Coffee,Kerala,Kalpetta,20130409,11800.0,0.5
4,4,Coffee,Kerala,Kalpetta,20130410,11900.0,0.5


In [66]:
df_concat = df_concat.dropna()

In [74]:
#converting 'Report Date' from int to date format  
df_concat["date"] = pd.to_datetime(df_concat['Report Date'].astype('str'), format = '%Y%m%d')

#calculating the weeek number of the year based on date
df_concat['weeks'] = df_concat["date"].dt.week

#selecting Onion data
df = df_concat[df_concat['Commodity'] == 'Onion']

In [75]:
# grouby 
df = df.groupby(["State", "Market Center", 'weeks'], as_index = False)

In [76]:
# aggregating weekly Prices and Quantity by taking Median of Prices n Quantities in a week
Pweekly = df["Modal Prices(in Rs/quintals)"].agg({'Median Price': np.median})
Qweekly = df["Arrival_Tonnes_Calc"].agg({'Median Quantity': np.median})

In [77]:
# taking log of Price and Quantity
Pweekly['Plog'] = np.log(Pweekly['Median Price'])
Qweekly['Qlog'] = np.log(Qweekly['Median Quantity'])

In [78]:
Qweekly.head()

,State,Market Center,weeks,Median Quantity,Qlog
0,Andhra Pradesh,Alur,7,0.1,-2.302585
1,Andhra Pradesh,Alur,8,0.1,-2.302585
2,Andhra Pradesh,Alur,11,0.1,-2.302585
3,Andhra Pradesh,Alur,26,17.3,2.850707
4,Andhra Pradesh,Alur,27,26.3,3.269569


In [79]:
Pweekly.head()

,State,Market Center,weeks,Median Price,Plog
0,Andhra Pradesh,Alur,7,537.5,6.286929
1,Andhra Pradesh,Alur,8,375.0,5.926926
2,Andhra Pradesh,Alur,11,400.0,5.991465
3,Andhra Pradesh,Alur,26,1523.0,7.328437
4,Andhra Pradesh,Alur,27,277.0,5.624018


In [80]:
# importing packages for econometric analysis
import statsmodels.api as sm          

In [81]:
#combing two dataframes
df_reg = pd.merge(Pweekly, Qweekly, on= ['State', 'Market Center', 'weeks'])

In [82]:
df_reg.head()

,State,Market Center,weeks,Median Price,Plog,Median Quantity,Qlog
0,Andhra Pradesh,Alur,7,537.5,6.286929,0.1,-2.302585
1,Andhra Pradesh,Alur,8,375.0,5.926926,0.1,-2.302585
2,Andhra Pradesh,Alur,11,400.0,5.991465,0.1,-2.302585
3,Andhra Pradesh,Alur,26,1523.0,7.328437,17.3,2.850707
4,Andhra Pradesh,Alur,27,277.0,5.624018,26.3,3.269569


In [73]:
#running regression analysis
trainX = df_reg["Plog"]
trainY = df_reg["Qlog"]

lX = sm.add_constant(trainX)
lm1 = sm.OLS(trainY,lX).fit()

In [83]:
print(lm1.params)

const    6.563333
Plog    -0.636871
dtype: float64


the price elasticity of demand for Onion across India is -0.636871

In [84]:
lm1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Qlog   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     1103.
Date:                Tue, 23 Jul 2019   Prob (F-statistic):          2.60e-237
Time:                        08:55:35   Log-Likelihood:                -60833.
No. Observations:               28257   AIC:                         1.217e+05
Df Residuals:                   28255   BIC:                         1.217e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.5633      0.145     45.299      0.000       6.279       6.847
Plog          -0.6369      0.019    -33.213      0.000      -0.674      -0.599
==============================================================================
Omnibus:                      310.670   Durbin-Watson:                   0.127
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              325.366
Skew:                           0.245   Prob(JB):                     2.23e-71
Kurtosis:                       3.191   Cond. No.                         89.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

the coefficients are significantly different from 0
as the t-statistic is significant

In [85]:
# running 2nd regression for State of Delhi
df_reg2 = df_reg[df_reg['State'] =='NCT of Delhi']
df_reg2.head()

,State,Market Center,weeks,Median Price,Plog,Median Quantity,Qlog
13371,NCT of Delhi,Azadpur,1,1271.0,7.147559,737.30,6.602995
13372,NCT of Delhi,Azadpur,2,1208.0,7.096721,893.80,6.795482
13373,NCT of Delhi,Azadpur,3,1121.0,7.021976,734.05,6.598577
13374,NCT of Delhi,Azadpur,4,1105.0,7.007601,567.50,6.341241
13375,NCT of Delhi,Azadpur,5,1097.0,7.000334,573.70,6.352107


In [64]:
trainX2 = df_reg2["Plog"]
trainY2 = df_reg2["Qlog"]

lX2 = sm.add_constant(trainX2)
l2 = sm.OLS(trainY2,lX2).fit()
l2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Qlog   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     2.931
Date:                Mon, 22 Jul 2019   Prob (F-statistic):             0.0890
Time:                        16:11:32   Log-Likelihood:                -235.96
No. Observations:                 150   AIC:                             475.9
Df Residuals:                     148   BIC:                             481.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.4323      1.193      7.068      0.000       6.075      10.790
Plog          -0.2711      0.158     -1.712      0.089      -0.584       0.042
==============================================================================
Omnibus:                       28.832   Durbin-Watson:                   0.182
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               38.836
Skew:                          -1.221   Prob(JB):                     3.69e-09
Kurtosis:                       3.499   Cond. No.                         95.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""